In [150]:
import pandas as pd
import numpy as np

In [151]:
df_cases = pd.read_csv('Resources/Data_Clean_Cases.csv')
df_cases.head()

,Unnamed: 0,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN


In [152]:
df_cases["date"].min()

'2020-01-01'

In [153]:
 # question 1)  speed of vaccination rollout by country gdp?
df_cases.columns
df_gdp = df_cases[["location","gdp_per_capita"]].groupby("location").max()

df_gdp.reset_index(level=0, inplace=True)
print(df_gdp.describe())

       gdp_per_capita
count      179.000000
mean     18776.122073
std      19687.204490
min        661.240000
25%       4338.764000
50%      12236.706000
75%      26792.862500
max     116935.600000


In [154]:
# create 4 gdp per capita groups based on 25,50,75 percentiles

df_gdp.loc[df_gdp["gdp_per_capita"] < 4000, 'gdpgrp'] = 'group 1'
df_gdp.loc[(df_gdp["gdp_per_capita"] >= 4000) & (df_gdp["gdp_per_capita"] < 12000), 'gdpgrp'] = 'group 2'
df_gdp.loc[(df_gdp["gdp_per_capita"] >= 12000) & (df_gdp["gdp_per_capita"] < 26000), 'gdpgrp'] = 'group 3'
df_gdp.loc[df_gdp["gdp_per_capita"] >= 26000, 'gdpgrp'] = 'group 4'
df_gdp    

,location,gdp_per_capita,gdpgrp
0,Afghanistan,1803.987,group 1
1,Albania,11803.431,group 2
2,Algeria,13913.839,group 3
3,Andorra,NaN,NaN
4,Angola,5819.495,group 2
...,...,...,...
181,Venezuela,16745.022,group 3
182,Vietnam,6171.884,group 2
183,Yemen,1479.147,group 1
184,Zambia,3689.251,group 1


In [155]:
# retrieving first covid case start date by country and saving to dataframe
covid_start_date = df_cases.loc[df_cases["total_cases"]>0,(["date","location"])].groupby("location")["date"].min()
covid_start_date_df = pd.DataFrame(covid_start_date)
covid_start_date_df
covid_start_date_df.reset_index(level=0, inplace=True)

In [156]:
# retrieving new_vaccination starting date by country and saving to dataframe
vaccination_start_date = df_cases.loc[df_cases["new_vaccinations"]>0,(["date","location"])].groupby("location")["date"].min()
vaccination_start_date
vaccination_start_date_df = pd.DataFrame(vaccination_start_date)


In [157]:
# merging covid_start_date_df to vaccination_start_date_df
covid_and_vax_dates = pd.merge(covid_start_date_df,vaccination_start_date_df, how="left", on="location", suffixes=("_covid_start","_first_vax"))
covid_and_vax_dates

,location,date_covid_start,date_first_vax
0,Afghanistan,2020-02-24,2021-05-27
1,Albania,2020-03-09,2021-01-13
2,Algeria,2020-02-25,2021-01-30
3,Andorra,2020-03-02,2021-04-05
4,Angola,2020-03-20,NaN
...,...,...,...
181,Venezuela,2020-03-14,NaN
182,Vietnam,2020-01-23,2021-03-08
183,Yemen,2020-04-10,NaN
184,Zambia,2020-03-18,2021-04-15


In [158]:
df_cases["%vaccinated"] = df_cases["people_fully_vaccinated"]/df_cases["population"]*100

In [159]:
df_cases[["location","%vaccinated"]]

,location,%vaccinated
0,Afghanistan,NaN
1,Afghanistan,NaN
2,Afghanistan,NaN
3,Afghanistan,NaN
4,Afghanistan,NaN
...,...,...
111064,Zimbabwe,16.391147
111065,Zimbabwe,16.433017
111066,Zimbabwe,16.574945
111067,Zimbabwe,16.625806


In [160]:
# finding latest figure of %vaccinated by country for heatmap
test = df_cases.loc[df_cases["%vaccinated"]>0,(["iso_code","location","date","%vaccinated"])].groupby(["location","iso_code"])[["%vaccinated"]].max()

#finding first row of 25% vaccinated by country
vax25 = df_cases.loc[df_cases["%vaccinated"]>25,(["location","date","%vaccinated"])].groupby("location")["date"].min()

#finding first row of 50% vaccinated by country
vax50 = df_cases.loc[df_cases["%vaccinated"]>50,(["location","date","%vaccinated"])].groupby("location")["date"].min()

#finding first row of 75% vaccinated by country
vax75 = df_cases.loc[df_cases["%vaccinated"]>75,(["location","date","%vaccinated"])].groupby("location")[["date"]].min()

In [161]:
# merging vaccination dfs to covid_and_vax_dates df
start_25_df = pd.merge(covid_and_vax_dates, vax25, how="left", on="location")
start_25_50_df = pd.merge(start_25_df, vax50, how="left", on="location", suffixes=("_25pc","_50pc"))
start_25_50_df = start_25_50_df.rename(columns={"date":"date_50pc"})
start_25_50_75_df = pd.merge(start_25_50_df, vax75, how="left", on="location")


In [162]:
start_25_50_75_df = start_25_50_75_df.rename(columns={"date":"date_75pc"})
start_25_50_75_df

,location,date_covid_start,date_first_vax,date_25pc,date_50pc,date_75pc
0,Afghanistan,2020-02-24,2021-05-27,NaN,NaN,NaN
1,Albania,2020-03-09,2021-01-13,2021-09-16,NaN,NaN
2,Algeria,2020-02-25,2021-01-30,NaN,NaN,NaN
3,Andorra,2020-03-02,2021-04-05,2021-06-21,2021-08-15,NaN
4,Angola,2020-03-20,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
181,Venezuela,2020-03-14,NaN,NaN,NaN,NaN
182,Vietnam,2020-01-23,2021-03-08,NaN,NaN,NaN
183,Yemen,2020-04-10,NaN,NaN,NaN,NaN
184,Zambia,2020-03-18,2021-04-15,NaN,NaN,NaN


In [163]:
start_25_50_75_df_gdp_groups = pd.merge(start_25_50_75_df, df_gdp, how="left", on="location")
start_25_50_75_df_gdp_groups

,location,date_covid_start,date_first_vax,date_25pc,date_50pc,date_75pc,gdp_per_capita,gdpgrp
0,Afghanistan,2020-02-24,2021-05-27,NaN,NaN,NaN,1803.987,group 1
1,Albania,2020-03-09,2021-01-13,2021-09-16,NaN,NaN,11803.431,group 2
2,Algeria,2020-02-25,2021-01-30,NaN,NaN,NaN,13913.839,group 3
3,Andorra,2020-03-02,2021-04-05,2021-06-21,2021-08-15,NaN,NaN,NaN
4,Angola,2020-03-20,NaN,NaN,NaN,NaN,5819.495,group 2
...,...,...,...,...,...,...,...,...
181,Venezuela,2020-03-14,NaN,NaN,NaN,NaN,16745.022,group 3
182,Vietnam,2020-01-23,2021-03-08,NaN,NaN,NaN,6171.884,group 2
183,Yemen,2020-04-10,NaN,NaN,NaN,NaN,1479.147,group 1
184,Zambia,2020-03-18,2021-04-15,NaN,NaN,NaN,3689.251,group 1


In [165]:
# calculate days to first vaccination
pd.to_datetime(start_25_50_75_df_gdp_groups["date_first_vax"]) - pd.to_datetime(start_25_50_75_df_gdp_groups["date_covid_start"])

0     458 days
1     310 days
2     340 days
3     399 days
4          NaT
        ...   
181        NaT
182   410 days
183        NaT
184   393 days
185   341 days
Length: 186, dtype: timedelta64[ns]